In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import shapely

import os
import sys

sys.path.append("../scripts")
from data import SmallPatchesDataset, get_filenames
from models import AutoEncoder, DEC

# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
n_cities_used = 100

# remove patches that are have small intersection with block
intersection_threshold = 0.25

# remove patches from blocks that have more than a threshold 
patches_count_max = 50

In [3]:
blocks_df = gpd.read_file("../data/census_blocks_patches_v2.geojson")

# filtering to test with some blocks, selecting the cities with the biggest number of blocks
biggest_blocks = blocks_df.groupby(["state", "county"]).agg("count").reset_index().iloc[:, 0:3].sort_values("tract", ascending = False).head(n_cities_used)
blocks_df = blocks_df[(blocks_df.state.isin(biggest_blocks.state) & blocks_df.county.isin(biggest_blocks.county))]

# cleaning blocks with missing data
blocks_df = blocks_df[blocks_df.mhi > 0]
blocks_df = blocks_df.dropna()
blocks_df = blocks_df[blocks_df.patches_relation.apply(len) > 0]

blocks_df["area_km2"] = blocks_df['geometry'].to_crs({'proj':'cea'}).area / 10**6
blocks_df["density"] = blocks_df["pop"] / blocks_df["area_km2"]

In [4]:
def clean_patches_relation(s):
    s = s.split("\n")
    s = dict([x.split(":") for x in s])
    filenames = []
    data = []
    for key, value in s.items():
        value = value.split(" ")
        idx = np.array([float(v) for v in value[0].split(",")])
        ratio = np.array([float(v) for v in value[1].split(",")])
        idx = idx[ratio > intersection_threshold]
        ratio = ratio[ratio > intersection_threshold]
        for i in range(len(idx)):
            data.append([idx[i], ratio[i]])
            filenames.append(key)
    data = np.array(data)
    if len(filenames) > patches_count_max:
        selected = np.random.choice(
            len(filenames),
            size=patches_count_max,
            replace=False,
            p=data[:, 1] / data[:, 1].sum(),
        )
        data = data[selected, :]
        filenames = [filenames[i] for i in selected]
    return [filenames, data]

blocks_df["clean_patches_relation"] = blocks_df.patches_relation.apply(
    clean_patches_relation
)
blocks_df["n_patches"] = blocks_df["clean_patches_relation"].apply(
    lambda x: x[1].shape[0]
)
blocks_df = blocks_df[blocks_df.n_patches > 0]

In [5]:
blocks_df = blocks_df.reset_index(drop = True)

In [6]:
patches_blocks = {}
for i, row in blocks_df.iterrows():
    relation_list = row["patches_relation"].strip(" ").split(" ")
    relation_list = [x.split(";") for x in relation_list]
    relation_list = row["clean_patches_relation"][0]
    idx = row["clean_patches_relation"][1][:, 0]
    files = [f"{relation_list[j]} {int(idx[j])}" for j in range(len(relation_list))]
    for file in files:
        if file in patches_blocks.keys():
            patches_blocks[file].append(i)
        else:
            patches_blocks[file] = [i]
print(f"Number of unique patches: {len(patches_blocks.keys())}")

Number of unique patches: 2214882


In [7]:
# get cluster of patches

def get_model(dec = False, k = 10, latent_dim = 128):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = AutoEncoder(
        latent_dim = latent_dim,
        encoder_arch = "resnet50_small_patch",
        decoder_layers_per_block = [3] * 5
    )
    if dec:
        model = DEC(
            n_clusters = k,
            embedding_dim = latent_dim,
            encoder = model.encoder
        )
        model.load_state_dict(torch.load(f"../models/DEC_resnet50_small_{latent_dim}_k_{k}/model.pt"))
    else:
        model.load_state_dict(torch.load(f"../model/AE_resnet50_small_{latent_dim}/model.pt"))
    model.to(device)
    return model

k = 30
latent_dim = 128
model = get_model(dec = True, k = k, latent_dim = latent_dim)

In [8]:
def get_clusters(dl, model):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    clusters = []
    clusters_distance = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dl):
            batch = batch.to(device)
            
            c = model(batch).detach().cpu().numpy()
            d = model.centroids_distance(batch).detach().cpu().numpy()
            clusters.append(c)
            clusters_distance.append(d)

    clusters = np.concatenate(clusters).argmax(axis = 1)
    clusters_distance = np.concatenate(clusters_distance)
    return clusters, clusters_distance


In [9]:
filenames = list(patches_blocks.keys())
dataset = SmallPatchesDataset(filenames)
dl = torch.utils.data.DataLoader(dataset, batch_size = 300)
clusters, clusters_distance = get_clusters(dl, model)

100%|██████████| 7383/7383 [2:46:05<00:00,  1.35s/it]  


## Comparison of all methods

In [10]:
# functions to generate features from model output
def count_of_patches_cluster():
    x = np.zeros((blocks_df.shape[0], k))
    for i, (file, cluster) in enumerate(zip(filenames, clusters)):
        for b in patches_blocks[file]:
            x[b, cluster] += 1
    x = pd.DataFrame(x, columns = [f"cluster_{i}" for i in range(k)])
    x = x.loc[:, x.sum(axis = 0) > 0]
    return x

def fraction_of_patches_cluster():
    x = np.zeros((blocks_df.shape[0], k))
    for i, (file, cluster) in enumerate(zip(filenames, clusters)):
        for b in patches_blocks[file]:
            x[b, cluster] += 1
    x_sum = x.sum(axis = 1)
    x = x / x_sum[:, None]
    x = pd.DataFrame(x, columns = [f"cluster_{i}" for i in range(k)])
    x = x.loc[:, x.sum(axis = 0) > 0]
    x["count"] = x_sum
    return x

def distance_of_patches_cluster():
    x = np.zeros((blocks_df.shape[0], k))
    for i, (file, distances) in enumerate(zip(filenames, clusters_distance)):
        for b in patches_blocks[file]:
            x[b] += distances
    x_sum = blocks_df.n_patches.values.reshape(-1)
    x = x / x_sum[:, None]
    x = pd.DataFrame(x, columns = [f"cluster_{i}" for i in range(k)])
    x = x.loc[:, x.sum(axis = 0) > 0]
    x["count"] = x_sum
    return x

In [11]:
def get_x(method):
    if method == "count":
        return count_of_patches_cluster()
    elif method == "fraction":
        return fraction_of_patches_cluster()
    elif method == "distance":
        return distance_of_patches_cluster()

In [12]:
def eval(clf, x_train, y_train, x_test, y_test):
    mae_train = mean_absolute_error(y_train, clf.predict(x_train))
    r2_train = r2_score(y_train, clf.predict(x_train))
    
    mae_test = mean_absolute_error(y_test, clf.predict(x_test))
    r2_test = r2_score(y_test, clf.predict(x_test))
    return r2_train, r2_test

In [13]:
def grid_search_rf(x_train, y_train, x_test, y_test):
    rf = RandomForestRegressor()
    parameters = {
        "n_estimators": [10, 100, 1000],
        "max_depth": [10, 100],
        #"min_samples_split": [2, 10, 100],
    }
    clf = GridSearchCV(rf, parameters, n_jobs=-1)
    clf.fit(x_train, y_train)
    return eval(clf, x_train, y_train, x_test, y_test)

In [14]:
class MLP(nn.Module):
    def __init__(self, dims):
        super(MLP, self).__init__()
        self.layers = []
        for in_dim, out_dim in zip(dims[:-1], dims[1:]):
            self.layers.append(nn.Linear(in_dim, out_dim))
            if out_dim != dims[-1]:
                self.layers.append(nn.ReLU())
        self.layers = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.layers(x)

    def predict(self, x):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.eval()
        if type(x) == pd.DataFrame:
            x_ = torch.from_numpy(x.values)
        elif type(x) == np.ndarray:
            x_ = torch.from_numpy(x)
            
        with torch.no_grad():
            x_ = x_.to(device)
            y = self.layers(x_)
            return y.detach().cpu().numpy()


In [15]:

def train_mlp(model, dl_train, dl_test):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    test_loss = []
    for i in range(100):
        iter_loss = 0
        for x, y in dl_train:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            iter_loss += loss.item()

        if i % 3 == 0:
            iter_loss = 0
            with torch.no_grad():
                for x, y in dl_test:
                    x, y = x.to(device), y.to(device)
                    y_pred = model(x)
                    loss = criterion(y_pred, y)
                    iter_loss += loss.item()
                test_loss.append(iter_loss)

            if i > 10 and test_loss[-1] > test_loss[-2]:
                break        

def grid_search_mlp(x_train, y_train, x_test, y_test):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    idx_train_, idx_val = train_test_split(np.arange(x_train.shape[0]), test_size = 0.2, random_state = 0)
    x_val_, y_val_ = x_train.values[idx_val, :], y_train[idx_val]
    x_train_, y_train_ = x_train.values[idx_train_, :], y_train[idx_train_]
    x_test_, y_test_ = x_test.values, y_test
    scaler = StandardScaler()
    x_train_ = scaler.fit_transform(x_train_)
    x_val_ = scaler.transform(x_val_)
    x_test_ = scaler.transform(x_test_)
    dl_train_ = DataLoader(TensorDataset(torch.tensor(x_train_), torch.tensor(y_train_.reshape(-1, 1))), batch_size = 128)
    dl_val = DataLoader(TensorDataset(torch.tensor(x_val_), torch.tensor(y_val_).reshape(-1, 1)), batch_size = 128)

    best_r2 = -np.inf
    best_model = None
    for dims in [[x_train.shape[1], 32, 64, 32, 1], [x_train.shape[1], 64, 256, 32, 1], [x_train.shape[1], 64, 512, 128, 1]]:
        model_1 = MLP(dims)
        model_1.to(device, dtype = torch.double)
        train_mlp(model_1, dl_train_, dl_val)
        r2_train, r2_test = eval(model_1, x_train_, y_train_, x_val_, y_val_)

        if r2_test > best_r2:
            best_r2 = r2_test
            best_model = model_1
    
    return eval(model_1, x_train_, y_train_, x_test_, y_test_)

In [16]:
idx_train, idx_test = train_test_split(np.arange(blocks_df.shape[0]), test_size = 0.2, random_state = 0)
df_results = []
for method in ["count", "fraction", "distance"]:
    x = get_x(method)
    x_train, x_test = x.loc[idx_train, :], x.loc[idx_test, :]
    for target in ["mhi", "ed_attain", "pop"]:
        y_train, y_test = blocks_df[target].values[idx_train], blocks_df[target].values[idx_test] 
        
        for regression in ["mlp", "rf"]:
            if regression == "rf":
                r2_train, r2_test = grid_search_rf(x_train, y_train, x_test, y_test)
            elif regression == "mlp":
                r2_train, r2_test = grid_search_mlp(x_train, y_train, x_test, y_test)

            print(f"method: {method}, target: {target}, regression: {regression}, r2_train: {r2_train}, r2_test: {r2_test}")
            df_results.append([method, target, regression, r2_train, r2_test])

method: count, target: mhi, regression: mlp, r2_train: 0.041799968292348444, r2_test: 0.03776554291722556
method: count, target: mhi, regression: rf, r2_train: 0.13762130340900602, r2_test: 0.03697626731578163
method: count, target: ed_attain, regression: mlp, r2_train: 0.0533397636650923, r2_test: 0.035623990381802195
method: count, target: ed_attain, regression: rf, r2_train: 0.14967380191264845, r2_test: 0.04107725952430075
method: count, target: pop, regression: mlp, r2_train: 0.057335305692281735, r2_test: 0.04856173654560303
method: count, target: pop, regression: rf, r2_train: 0.17789558257982452, r2_test: 0.043772433577050296
method: fraction, target: mhi, regression: mlp, r2_train: 0.045877053527016876, r2_test: 0.03960599833934242
method: fraction, target: mhi, regression: rf, r2_train: 0.14712944318986187, r2_test: 0.035981931920656796
method: fraction, target: ed_attain, regression: mlp, r2_train: 0.05951466433903396, r2_test: 0.042960292883652995
method: fraction, target: 